In [1]:
import os
import pandas as pd

In [2]:
def merge_csv_files(folder_path, output_file):
    # List all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    # Initialize an empty DataFrame to store merged data
    merged_df = pd.DataFrame()
    
    # Read each CSV file and merge into the DataFrame
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")

In [3]:
# Example usage:
folder_path = '/path/to/csv/files'  # Specify the folder containing CSV files
output_file = 'merged_data.csv'      # Specify the name of the output merged CSV file

merge_csv_files(folder_path, output_file)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/csv/files'